The main solving big ridge regression problems

In [1]:
workspace()

In [2]:
using Plots

INFO: Recompiling stale cache file /home/ROCQ/sierra/rgower/.julia/lib/v0.5/FixedSizeArrays.ji for module FixedSizeArrays.


INFO: Recompiling stale cache file /home/ROCQ/sierra/rgower/.julia/lib/v0.5/PlotUtils.ji for module PlotUtils.
INFO: Recompiling stale cache file /home/ROCQ/sierra/rgower/.julia/lib/v0.5/PlotThemes.ji for module PlotThemes.
INFO: Recompiling stale cache file /home/ROCQ/sierra/rgower/.julia/lib/v0.5/Showoff.ji for module Showoff.
INFO: Recompiling stale cache file /home/ROCQ/sierra/rgower/.julia/lib/v0.5/StatsBase.ji for module StatsBase.
INFO: Recompiling stale cache file /home/ROCQ/sierra/rgower/.julia/lib/v0.5/NaNMath.ji for module NaNMath.
INFO: Recompiling stale cache file /home/ROCQ/sierra/rgower/.julia/lib/v0.5/FileIO.ji for module FileIO.


In [3]:
using StatsBase
using JLD  
include("../src/BigRidge.jl")

INFO: Recompiling stale cache file /home/ROCQ/sierra/rgower/.julia/lib/v0.5/JLD.ji for module JLD.


plot_outputs_Plots (generic function with 1 method)

In [4]:
#Basic parameters
## Basic parameters
maxiter=10^8;
max_time = 10;
max_epocs = 100;
printiters = true;
exacterror =true; 
tol = 10.0^(-5.0);
restol = 10.0^(-5.0);
skip_error_calculation = 25;   # number of iterations where error is not calculated (to save time!) ceil((m)^(1/2))

In [5]:
dataset = "a9a"
prob = defineProb(dataset);

../data/a9a.jld


In [6]:
norm(prob.A*prob.xsol - prob.b)

4.459713089533306e-11

In [7]:
options = MyOptions(0,"uni",tol,restol,maxiter,skip_error_calculation,max_time,printiters,exacterror) 
OUTPUTS = [];

In [8]:
# Testing Rademarcher Sketch
method_name = "rademacher";# ROS
options.sketchsize =5;
options.maxiter=100;
output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];

rademacher
-------------------
It   | Error% | Residual |  Time   
-------------------
 40  | 207.45  |  2.08  | 0.3006 
 50  | 144.48  |  1.54  | 0.3017 
 75  | 138.66  |  1.06  | 0.3035 
100  | 131.03  |  0.41  | 0.3054 


In [13]:
x = rand(prob.n)
dot(x-prob.xsol,prob.A*(x-prob.xsol))


445972.34720663883

In [ ]:
# Testing Hadamard Sketch
method_name = "Hadamard";# ROS
options.sketchsize =prob.n; # Int(floor(log2(prob.n)));
options.maxiter=100;
options.skip_error_calculation =1;
output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];

In [78]:
# Testing CG 
options.max_time = 2; options.restol = 10.0^(-8); 
options.skip_error_calculation =25;
method_name = "CG";# gradient descent
output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];

CG
-------------------
It   | Error% | Residual |  Time   
-------------------
 25  | 35.13  |  0.07  | 0.0087 
 50  | 10.62  |  0.02  | 0.0089 
 75  | 6.74  |  0.00  | 0.0092 
100  | 5.59  |  0.00  | 0.0095 


In [ ]:
# Testing grad descent
options.max_time = 2; options.restol = 10.0^(-8); options.maxiter=1000;
method_name = "grad";# gradient descent
output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];

In [ ]:
# Testing CD 
options.max_time = 10; options.restol = 10.0^(-8); options.maxiter=10000;
method_name = "CD";# gradient descent
options.sketchsize = 11#floor((prob.n)^(1/2));
output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];

In [ ]:
savename = replace(prob.name, r"[\/]", "-");
save("../data/$(savename)-OUTPUTS.jld", "OUTPUTS", OUTPUTS)

In [79]:
#using GR
using Plots
pgfplots()
plot_outputs_Plots(OUTPUTS,savename)

Output(100,1000,[0.0 0.00842004 0.0162041 0.0234429 0.0314144],[1.0 1.52037 1.275 1.21623 1.11559],[1.0 0.0132052 0.00394501 0.00267697 0.00237649],"rademacher","max_iter")
rademacher
Output(100,31119,[0.0 0.00882264 0.0108304 0.0120699 0.0187137],[1.0 0.351309 0.106186 0.0674324 0.0559211],[1.0 0.00074717 0.000221425 3.43877e-5 8.87998e-6],"CG","max_iter")
CG
Output(100,31119,[0.0 0.00865575 0.00894916 0.00920735 0.00946474],[1.0 0.351309 0.106186 0.0674324 0.0559211],[1.0 0.00074717 0.000221425 3.43877e-5 8.87998e-6],"CG","max_iter")
CG
a9a


In [ ]:
OUTPUTS[4]

In [ ]:
# Loading saved problem data
probname = "w8a"#"gisette_scale-ridge";  
test = load("../data/$(probname)-OUTPUTS.jld")
OUTPUTS = test["OUTPUTS"];